## Minimize and equilibrate
 - Minimize
 - Equilibrate
 
 Sanity checks along the way.
 
 Protocol files are copied from https://github.com/intbio/gmx_protocols/tree/master/amber into
Protocols folder.
If others are needed modify then in the folder.

Theoretically this notebook could be exectuted in a single step, but it is unclear currently how stable the execution of the notebook is if you close browser window or connection in interrupted.

An alternative is to run from a persistent (opened via X2Go or using Tmux) terminal as

```
jupyter nbconvert --ExecutePreprocessor.timeout=72000 --ExecutePreprocessor.kernel_name=python3 --to notebook --execute 2_min_equil.ipynb --output 2_min_equil.ipynb
```




For TMUX on newton cheatsheet
```
tmux
module load conda3
source activate moldyn
Ctr+b d - detach
tmux ls
tmux attach -t 0
```


All cells that deal with transfering simulations to Supercomputer are currently inactive - set to test mode!!!




In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from myimports import *
display(Markdown(descr))

_ColormakerRegistry()

Folder set to   :  gmx_template_for_nmr
Project name    :  gmx_template_for_nmr
SSH host set to : lomo2
GMX cmd set with: module load gromacs/2020.1_nompi
Force field     : amber14sb_parmbsc1_cufix
Force field URL : https://api.github.com/repos/intbio/gromacs_ff/contents/amber14sb_parmbsc1_cufix.ff
Run profile     : lomo2_volta2.sh



# MD simulations of NCP with histone tails for NMR parameters calculation 
- AMBER14SB force field
- 150 mM NaCl
- box 2nm


### 1_Minimization - steepest descent gradient method 10000 steps with POSRED constraints

#### Prepare tpr files using GROMPP
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-grompp.html

In [3]:
%%bash -s "$set_gmx_cmd"
$1
#Prepare system
rm -rf GMX_run_prep
mkdir -p GMX_run_prep
gmx grompp -maxwarn 1 -f MDProtocols/1_minim.mdp -c GMX_system_prep/init_solv_ions.pdb -p GMX_system_prep/topol.top \
-o GMX_run_prep/1_minim.tpr -r GMX_system_prep/init_solv_ions.pdb -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_1_minim.mdp


Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 256x256x256, spacing 0.115 0.115 0.115
This run will generate roughly 710 Mb of data


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Run: Option 1 - locally
Note: try to run on a small number of CPUs with no GPUs, because system in non-equilibrated.

In [4]:
%%bash -s "$set_gmx_cmd" "$gmx_eq_cmd"
$1
cd GMX_run_prep
gmx mdrun -ntomp 20 -ntmpi 1 -deffnm 1_minim 2> 1_minim.sterr  # if you want to run locally, nt is the number of threads
echo "Outputting gromacs error log"
cat 1_minim.sterr

Outputting gromacs error log
                      :-) GROMACS - gmx mdrun, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           an

(Un)Loading module for modern CPU


### Troubleshooting exploding trajectories !!! (if needed)
------

In [5]:
#First steps often require troubleshooting and looking in vmd at the structure.
#To do it you'll need to change MD_Protocols/1_minim.mdp to decrease number of steps to 20 or 100, and set output for every step.
#Let's generate useful commands that can be used on your local machine.
# See if there are any real, problems or clashed, LINCS warnings are usually sensitive to that.
#If there are still unexplainable problems with LINCS, try to change constrains = None in 1_minim.mdp file, usually this helps.
dir=!pwd
newton_user=!whoami
!cp VMD_scripts/min_troubleshoot.tcl GMX_run_prep/
!cp GMX_system_prep/init_solv_ions.pdb GMX_run_prep
with open('GMX_run_prep/1_minim.sterr', "r") as file:
    counter=0
    line='1'
    atoms=[]
    while ((counter<1000) and line):
        line=file.readline()
        m=re.match("\s+(\d+)\s+\d+\s+\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+",line)
        if(m):
            atoms.append(m.groups()[0])
            counter+=1
        else:
            next
    atomstr=' '.join(atoms)
if(len(atoms)>0):
    !echo 'mol representation VDW' >> GMX_run_prep/min_troubleshoot.tcl
    !echo "mol selection index $atomstr" >> GMX_run_prep/min_troubleshoot.tcl
    !echo "mol addrep top" >> GMX_run_prep/min_troubleshoot.tcl
#!echo "$atomstr"
print("rm -r GMX_run_prep")
print("rsync -avz --exclude='step*' --exclude='\#*' %s@newton.bioeng.ru:%s/GMX_run_prep ."%(newton_user[0],dir[0]))
print("vmd -e GMX_run_prep/min_troubleshoot.tcl\n")

rm -r GMX_run
rsync -avz --exclude='step*' --exclude='\#*' alexsha@newton.bioeng.ru:/home/_shared/_projects/gmx_template/GMX_run .
vmd -e GMX_run/min_troubleshoot.tcl



------------


#### Run: Option 2 - sending and simulating on Lomonosov-2
Note: Usually it is easier just to run all the cells for local simulations below and wait some time, than to send to supercomputer since it requires manual work and monitoring. But examples of doing it on supercomputer are provided for the next two step for reference.
They are commented out (set to RAW content), so that this note book could be run entirely as a single piece of code locally.

In [6]:
deffnm='1_minim'

#### Analyze what we get

### 2_Equilibration 100 ps with postional restraints of 500 kJ/mol/A2 , and with 0.5 fs time step

In [5]:
%%bash -s "$set_gmx_cmd"
$1 
#--out out --err err
gmx grompp -maxwarn 1 -f MDProtocols/2_equil.mdp -c GMX_run_prep/1_minim.gro -p GMX_system_prep/topol.top \
-o GMX_run_prep/2_equil.tpr -v  -r GMX_system_prep/init_solv_ions.pdb -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_2_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298 K
Calculated rlist for 1x1 atom pair-list as 1.050 nm, buffer size 0.000 nm
Set rlist, assuming 4x4 atom pair-list, to 1.050 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 256x256x256, spacing 0.115 0.115 0.115
This run will generate roughly 283 Mb of data


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Run: Option 1 - locally

In [6]:
%%bash -s "$set_gmx_cmd" "$gmx_eq_cmd"
$1
cd GMX_run_prep
# first run shoulr be without notunepme to allow gromacs to tune it :)
$2 -deffnm 2_equil # if you want to run locally

(Un)Loading module for modern CPU
                      :-) GROMACS - gmx mdrun, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                        

#### Run: Option 2 - sending and simulating on Lomonosov-2
Note: Usually it is easier just to run all the cells for local simulations below and wait some time, than to send to supercomputer since it requires manual work and monitoring. But examples of doing it on supercomputer are provided for this step for reference. The same can be applied to all further step if needed - just copy the code.
The code here is commented out (cell type set to RAW content), so that this notebook could be run entirely as a single piece of code locally.

In [9]:
deffnm='2_equil'

#### Analyze what we get

### 3_Equilibration 200 ps with postional restraints of 50 kJ/mol/A2

In [7]:
%%bash -s "$set_gmx_cmd"
$1
#--out out --err err
gmx grompp -maxwarn 1 -f MDProtocols/3_equil.mdp -c GMX_run_prep/2_equil.gro -p GMX_system_prep/topol.top \
-o GMX_run_prep/3_equil.tpr -v  -r GMX_system_prep/init_solv_ions.pdb -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_3_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298 K
Calculated rlist for 1x1 atom pair-list as 1.080 nm, buffer size 0.030 nm
Set rlist, assuming 4x4 atom pair-list, to 1.050 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 224x224x224, spacing 0.119 0.119 0.119
This run will generate roughly 184 Mb of data


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Run: Option 1 - locally

In [8]:
%%bash -s "$set_gmx_cmd" "$gmx_eq_cmd"
$1
cd GMX_run_prep
$2 -deffnm 3_equil # if you want to run locally

(Un)Loading module for modern CPU
                      :-) GROMACS - gmx mdrun, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                        

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [9]:
deffnm='3_equil'

### 4_Equilibration 200 ps with postional restraints of 5 kJ/mol/A2

In [10]:
%%bash -s "$set_gmx_cmd"
$1
#--out out --err err
gmx grompp -maxwarn 1 -f MDProtocols/4_equil.mdp -c GMX_run_prep/3_equil.gro -p GMX_system_prep/topol.top \
-o GMX_run_prep/4_equil.tpr -v  -r GMX_system_prep/init_solv_ions.pdb -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_4_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298 K
Calculated rlist for 1x1 atom pair-list as 1.085 nm, buffer size 0.035 nm
Set rlist, assuming 4x4 atom pair-list, to 1.050 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 200x200x200, spacing 0.120 0.120 0.120
This run will generate roughly 184 Mb of data


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Run: Option 1 - locally

In [11]:
%%bash -s "$set_gmx_cmd" "$gmx_eq_cmd"
$1
cd GMX_run_prep
$2 -deffnm 4_equil # if you want to run locally

(Un)Loading module for modern CPU
                      :-) GROMACS - gmx mdrun, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                        

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [15]:
deffnm='4_equil'

### 5_Equilibration 200 ps with postional restraints of 0.5 kJ/mol/A2

In [12]:
%%bash -s "$set_gmx_cmd" 
$1
#--out out --err err
gmx grompp -maxwarn 1 -f MDProtocols/5_equil.mdp -c GMX_run_prep/4_equil.gro -p GMX_system_prep/topol.top \
-o GMX_run_prep/5_equil.tpr -v  -r GMX_system_prep/init_solv_ions.pdb -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_5_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298 K
Calculated rlist for 1x1 atom pair-list as 1.085 nm, buffer size 0.035 nm
Set rlist, assuming 4x4 atom pair-list, to 1.050 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 200x200x200, spacing 0.119 0.119 0.119
This run will generate roughly 184 Mb of data


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Run: Option 1 - locally

In [13]:
%%bash -s "$set_gmx_cmd" "$gmx_eq_cmd"
$1
cd GMX_run_prep
$2 -deffnm 5_equil # if you want to run locally

(Un)Loading module for modern CPU
                      :-) GROMACS - gmx mdrun, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                        

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [18]:
deffnm='5_equil'

### 6_Equilibration 200 ps unrestrained

In [14]:
%%bash -s "$set_gmx_cmd"
$1 
#--out out --err err
gmx grompp -maxwarn 1 -f MDProtocols/6_equil.mdp -c GMX_run_prep/5_equil.gro -p GMX_system_prep/topol.top \
-o GMX_run_prep/6_equil.tpr -v  -r GMX_system_prep/init_solv_ions.pdb -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_6_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298 K
Calculated rlist for 1x1 atom pair-list as 1.085 nm, buffer size 0.035 nm
Set rlist, assuming 4x4 atom pair-list, to 1.050 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 200x200x200, spacing 0.119 0.119 0.119
This run will generate roughly 184 Mb of data


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Run: Option 1 - locally

In [15]:
%%bash -s "$set_gmx_cmd" "$gmx_eq_cmd"
$1
cd GMX_run_prep
$2 -deffnm 6_equil # if you want to run locally

(Un)Loading module for modern CPU
                      :-) GROMACS - gmx mdrun, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                        

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [21]:
deffnm='6_equil'

# Saving final equilibrated results

As a result we have equlibrated system ready for production simulations.
We copy it to GMX_system_prep.

In [16]:
%%bash
cp GMX_run_prep/6_equil.gro GMX_system_prep/ #coordinates
cp GMX_run_prep/6_equil.cpt GMX_system_prep/ #This has also velocities





Optionally we can save all equilibrated files, if want to copy them somewhere and user later,
here we place into results folder the set of equilibrated files of step 6.


In [17]:
%%bash -s "$set_gmx_cmd"
$1
gmx trjconv -f GMX_run_prep/6_equil.gro -s GMX_run_prep/6_equil.tpr -o GMX_run_prep/6_equil.pdb << !
0
!

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for output
Selected 0: 'System'


(Un)Loading module for modern CPU
                     :-) GROMACS - gmx trjconv, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

In [18]:
%%bash
mkdir -p results
cp GMX_run_prep/6_equil* results/
cp GMX_system_prep/sys_ref.pdb results/

## 7_Production run 100 ns - for tails condencation 

In [19]:
%%bash -s "$set_gmx_cmd" 
$1
#--out out --err err
gmx grompp -maxwarn 2 -f MDProtocols/7_prod.mdp -c GMX_run_prep/6_equil.gro -p GMX_system_prep/topol.top \
-o GMX_run_prep/7_prod.tpr -v  -r GMX_run_prep/6_equil.gro -n GMX_system_prep/index.ndx -po GMX_run_prep/mdout_7_prod.mdp -t GMX_run_prep/6_equil.cpt


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298 K
Calculated rlist for 1x1 atom pair-list as 1.085 nm, buffer size 0.035 nm
Set rlist, assuming 4x4 atom pair-list, to 1.050 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 200x200x200, spacing 0.119 0.119 0.119


(Un)Loading module for modern CPU
                      :-) GROMACS - gmx grompp, 2020.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

### Start production run


### Transfer files to supercomputer if needed

In [2]:
run_profile = 'lomo2_compute_prep.sh'

In [30]:
%%bash -s "$run_profile" "$ssh_host" "$folder"
source Run_profiles/$1
create_dir $2 $3

Params: lomo2 gmx_template_for_nmr


+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+
|  Welcome to the Lomonosov-2 supercomputer!  |
+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+



In [37]:
!cp Run_scripts/* GMX_run_prep/

In [38]:
%%bash -s "$run_profile" "$ssh_host" "$folder"
source Run_profiles/$1
push_data $2 $3

Params: lomo2 gmx_template_for_nmr


+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+
|  Welcome to the Lomonosov-2 supercomputer!  |
+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+



### Launch job

In [33]:
%%bash -s "$run_profile" "$ssh_host" "$folder"
source Run_profiles/$1
launch_initial_job $2 $3

Params: lomo2 gmx_template_for_nmr
Submitted batch job 1494246


+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+
|  Welcome to the Lomonosov-2 supercomputer!  |
+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+



In [34]:
run_profile

'lomo2_compute_prep.sh'

In [40]:
%%bash -s "$run_profile" "$ssh_host" "$folder"
source Run_profiles/$1
relaunch $2 $3

Params: lomo2 gmx_template_for_nmr
Submitted batch job 1500753


+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+
|  Welcome to the Lomonosov-2 supercomputer!  |
+-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-+



In [79]:
!cat GMX_run_prep/6_equil.log | grep -i 'Performance:'
# (ns/day)    (hour/ns)

Performance:        7.057        3.401


# Get back data 

In [ ]:
%%bash -s "$run_profile" "$ssh_host" "$folder"
source Run_profiles/$1
pull_data $2 $3

# Save last frame 

In [84]:
u = mda.Universe('GMX_system_prep/sys_ref.pdb','big_data/sys_md_prep.xtc')
sel = u.select_atoms('protein or nucleic')
sel.write('GMX_system_prep/sys_condensed.pdb',frames=u.trajectory[-1:])

In [85]:
view_nucl('GMX_system_prep/sys_condensed.pdb')

NGLWidget()

# Trajectory processing 

In [81]:
from myimports import *

In [80]:
!mkdir -p tmp
!mkdir -p big_data

In [53]:
print(err)

(Un)Loading module for modern CPU
bash: line 3: ./2_trj_preprocessing_prep.sh: Permission denied



In [55]:
%%bash  -s "$set_gmx_cmd"  --err err --out out
$1
#The code below was collapsed to one shell file
bash ./2_trj_preprocessing_prep.sh 7_prod sys_md_prep 

In [58]:
view_nucl('GMX_system_prep/sys_ref.pdb','big_data/sys_md_prep.xtc', gui=True)

NGLWidget(max_frame=10502)

In [59]:
view_nucl('GMX_system_prep/sys_ref.pdb')

NGLWidget()